In [11]:
import pandas as pd
import json

# This section  below is for the purpose of tabularizing json and performing data validation from source to target

In [30]:
json = {
    "data": [
        {
            "MainId": 1111,
            "firstName": "Sherlock",
            "lastName": "Homes",
            "categories": [
                {
                    "CategoryID": 1,
                    "CategoryName": "Example"
                }
            ]
        },
        {
            "MainId": 122,
            "firstName": "James",
            "lastName": "Watson",
            "categories": [
                {
                    "CategoryID": 2,
                    "CategoryName": "Example2"
                }
            ]
        }
    ],
    "messages": [], 
    "success": 'True' 
}

## Function That Parses JSON And Creates A DataFrame To Be Tested

In [20]:
# Get the outer level meta data to input as a parameter in the json_normalize() function
def outer_flatten_list_with_df(dictionary):
    most_outer_list = []
    most_outer_df = pd.json_normalize(dictionary).T
    most_outer_df['NodePath'] = most_outer_df.index
    most_outer_list = []
    df_list = []
    droplist = []

    for i in range(len(most_outer_df)):
        ttype = most_outer_df.iloc[i, 0]
        if isinstance(ttype, dict) == True:
            most_outer_list.append(most_outer_df.index[i])
            droplist.append(most_outer_df.index[i])
        elif isinstance(ttype, list) == True:
            most_outer_list.append(most_outer_df.index[i])
            droplist.append(most_outer_df.index[i])
        elif isinstance(ttype, list) == True and len(ttype) == 0:
            inner_df = most_outer_df['NodePath'].iloc[i]
            inner_df['NodePath'].iloc[i] = key + '.' + inner_df
        elif isinstance(ttype, list) == True and len(ttype) == 0:
            inner_df['NodePath'].iloc[i] = key + '.' + inner_df['NodePath'].iloc[i]
            
    only_outer_df = most_outer_df.drop(droplist)
    df_list.append(only_outer_df)
    return [df_list, most_outer_list]

# Takes meta data from outer_flatten_list_with_df and parses one level down
def inner_flatten_list_with_df1(lst, dictionary):
    inner_layers_list = []
    df_list = []
    droplist = []

    if len(lst) == 0:
        return 'No values to parse'

    for key in lst:
        droplist = []
        if '.' in key:
            keylist = key.split('.')
            inner_df = pd.json_normalize(dictionary, record_path=keylist).T
            inner_df['NodePath'] = inner_df.index
        else:
            inner_df = pd.json_normalize(dictionary, record_path=key).T
            inner_df['NodePath'] = inner_df.index

        for i in range(len(inner_df)):
            ttype = inner_df.iloc[i, 0]
            if isinstance(ttype, dict) == True:
                # mol.append(most_outer_df.index[i])
                inner_layers_list.append(key + '.' + inner_df.index[i])
                droplist.append(inner_df.index[i])
            elif isinstance(ttype, list) == True:
                # mol.append(most_outer_df.index[i])
                inner_layers_list.append(key + '.' + inner_df.index[i])
                droplist.append(inner_df.index[i])
            elif isinstance(ttype, list) == True and len(ttype) == 0:
                inner_df['NodePath'].iloc[i] = key + '.' + inner_df['NodePath'].iloc[i]
            elif isinstance(ttype, list) == True and len(ttype) == 0:
                inner_df['NodePath'].iloc[i] = key + '.' + inner_df['NodePath'].iloc[i]
            else:
                inner_df['NodePath'].iloc[i] = key + '.' + inner_df['NodePath'].iloc[i]

        only_outer_df = inner_df.drop(droplist)
        df_list.append(only_outer_df)

    return [df_list, inner_layers_list]


# Takes meta data from outer_flatten_list_with_df and parses one level down
def inner_flatten_list_with_df2(lst, dictionary):
    inner_layers_list = []
    df_list = []
    droplist = []

    if len(lst) == 0:
        return 'No values to parse'

    for key in lst:
        droplist = []
        if '.' in key:
            keylist = key.split('.')
            inner_df = pd.json_normalize(dictionary, record_path=keylist).T
            inner_df['NodePath'] = inner_df.index
        else:
            inner_df = pd.json_normalize(dictionary, record_path=key).T
            inner_df['NodePath'] = inner_df.index

        for i in range(len(inner_df)):
            ttype = inner_df.iloc[i, 0]
            if isinstance(ttype, dict) == True:
                # mol.append(most_outer_df.index[i])
                inner_layers_list.append(key + '.' + inner_df.index[i])
                droplist.append(inner_df.index[i])
            elif isinstance(ttype, list) == True:
                # mol.append(most_outer_df.index[i])
                inner_layers_list.append(key + '.' + inner_df.index[i])
                droplist.append(inner_df.index[i])
            elif isinstance(ttype, list) == True and len(ttype) == 0:
                inner_df['NodePath'].iloc[i] = key + '.' + inner_df['NodePath'].iloc[i]
            elif isinstance(ttype, list) == True and len(ttype) == 0:
                inner_df['NodePath'].iloc[i] = key + '.' + inner_df['NodePath'].iloc[i]
            else:
                inner_df['NodePath'].iloc[i] = key + '.' + inner_df['NodePath'].iloc[i]

        only_outer_df = inner_df.drop(droplist)
        df_list.append(only_outer_df)

    return [df_list, inner_layers_list]


# Uses the three functions and creates a pattern that will parse down to the lowest level in a given JSON.
def json_dataset(dictionary):
    all_meta_data = []
    nested_list_of_dataframes = []

    outer_list = outer_flatten_list_with_df(dictionary)[1]
    outer_df = outer_flatten_list_with_df(dictionary)[0]

    all_meta_data.append(outer_list)
    nested_list_of_dataframes.append(outer_df)

    inner_flatten_list = inner_flatten_list_with_df1(outer_list, dictionary)[1]
    inner_flatten_df1 = inner_flatten_list_with_df1(outer_list, dictionary)[0]

    all_meta_data.append(inner_flatten_list)
    nested_list_of_dataframes.append(inner_flatten_df1)

    while len(inner_flatten_list) > 0:
        inner_flatten_list = inner_flatten_list_with_df2(inner_flatten_list, dictionary)[1]
        inner_flatten_df2 = inner_flatten_list_with_df2(inner_flatten_list, dictionary)[0]
        all_meta_data.append(inner_flatten_list)
        nested_list_of_dataframes.append(inner_flatten_df2)

    # if len(inner_flatten_list) > 0:
    #     inner_flatten_list = inner_flatten1(inner_flatten_list, dictionary)
    #     all_meta_data.append(inner_flatten_list)

    json_dataframe_list = []
    for dflist in nested_list_of_dataframes:
        for df in dflist:
            json_dataframe_list.append(df)

    deal_concatdf = pd.concat(json_dataframe_list[:-1]).sort_values(by='NodePath')
    sorted_json_concatdf = deal_concatdf.rename(columns={0: "Values"})
    json_concatdf = sorted_json_concatdf[['NodePath', 'Values', 1]]

    return json_concatdf


In [43]:
[json_dataset(json)]

[                 NodePath    Values       1
 MainId        data.MainId      1111     122
 firstName  data.firstName  Sherlock   James
 lastName    data.lastName     Homes  Watson
 success           success      True     NaN]

## Test every Combination From Source To Target And Bring Back ids OF Passes and Fails

In [48]:
sql_id_list2 = []
snowflake_id_list2 = []
match_list = []
sql_mismatch_list = []
snowflake_mismatch_list = []
mismatch_list = []

# This code is testing if the JSON has matching property name and loan number. Then it puts the IDs from source and target
# into separate lists, where the IDs are the same index between the two lists.
for i in range(len(parsed_json_sql_dataframe_list)):
    for j in range(len(parsed_json_snowflake_dataframe_list)):
        # if parsed_json_snowflake_dataframe_list[j].query('NodePath == "propertyName"').iloc[0, 0] \
        # == parsed_json_sql_dataframe_list[i].query('NodePath == "propertyName"').iloc[0, 0] and \
        # parsed_json_snowflake_dataframe_list[j].iloc[:, 1] == parsed_json_sql_dataframe_list[i].iloc[:, 1]:

        if len(parsed_json_snowflake_dataframe_list[j]) == len(parsed_json_sql_dataframe_list[i]) \
           and len(set(parsed_json_snowflake_dataframe_list[j].index) - set(parsed_json_sql_dataframe_list[i].index)) == 0 \
           and len(set(parsed_json_snowflake_dataframe_list[j].columns) - set(parsed_json_sql_dataframe_list[i].columns)) == 0:
            
            results = parsed_json_snowflake_dataframe_list[j].compare(parsed_json_sql_dataframe_list[i])

            # If the tests are a pass, then show the IDs from the table
            if len(results) == 0:
                sql_id_list2.append(sql_source_dataframe.iloc[i, 0])
                snowflake_id_list2.append(snowflake_target_dataframe.iloc[j, 4])
                match_list.append('Pass')
            if len(results) > 0:
                sql_mismatch_list.append(sql_source_dataframe.iloc[i, 0])
                snowflake_mismatch_list.append(snowflake_target_dataframe.iloc[j, 4])
                mismatch_list.append('Fail')


# Make Dataset Of All Test Combinations And Write Results To A File

In [50]:
#Create DataFrame for mismatches
mismatch_df = pd.DataFrame({
    'SQL_id_Column_Values': sql_mismatch_list,
    'Snowflake_INPUT_KEY_Column_Values': snowflake_mismatch_list,
    'Results': mismatch_list
})

# Create DataFrame for matches
match_df = pd.DataFrame({
    'SQL_id_Column_Values': sql_id_list2,
    'Snowflake_INPUT_KEY_Column_Values': snowflake_id_list2,
    'Results': match_list
})

# Combine match and mismatch DataFrames
cmm_individual_results_df = pd.concat([mismatch_df, match_df])
cmm_individual_results_df.index = np.arange(1, len(cmm_individual_results_df) + 1)

# Export the combined DataFrame to an Excel file
cmm_individual_results_df.to_excel('outbound/Cmm_Individual_Results.xlsx', index=False)


# Data Set Showing The Results By Unique Key For Snowflake Snowflake_Input_Key

In [51]:
# Filter and display unique 'Pass' results by Snowflake_DEAL_INPUT_KEY_Column_Values
unique_pass_df = cmm_individual_results_df.query('Results == "Pass"').drop_duplicates('Snowflake_INPUT_KEY_Column_Values')
unique_pass_df.index = np.arange(1, len(unique_pass_df) + 1)
unique_pass_df

# Filter and display unique 'Fail' results by Snowflake_DEAL_INPUT_KEY_Column_Values
unique_fail_df = cmm_individual_results_df.query('Results == "Fail"').drop_duplicates('Snowflake_INPUT_KEY_Column_Values')
unique_fail_df.index = np.arange(1, len(unique_fail_df) + 1)
unique_fail_df

## Get Ids Of JSON That Has Matching PropertyName and Loanid

In [49]:
sql_id_list = []
snowflake_id_list = []

# only use the results from this code when the amount of loans are 1:1 from SQL to Snowflake.

# This code is testing if the JSON has matching property name and loan number. Then it puts the IDs from source and target
# into separate lists, where the IDs are the same index between the two lists.
for i in range(len(parsed_json_snowflake_dataframe_list)):
    for j in range(len(parsed_json_sql_dataframe_list)):
        if parsed_json_snowflake_dataframe_list[i].query('NodePath == "propertyName"').iloc[0, 0] \
           == parsed_json_sql_dataframe_list[j].query('NodePath == "propertyName"').iloc[0, 0] and \
           parsed_json_snowflake_dataframe_list[i].iloc[i, 1] \
           == parsed_json_sql_dataframe_list[j].iloc[j, 1]:
            
            sql_id_list.append(sql_source_dataframe.iloc[j, 0])
            snowflake_id_list.append(snowflake_target_dataframe.iloc[i, -4])


# Query The SQL And Snowflake Tables For JSON By Id's

In [52]:
concat_sql_dataframe_list = []
concat_snowflake_dataframe_list = []

# Loops through the ID lists in parallel. The JSONs that are on the same rows of the IDs should match.
# The corresponding JSONs are then parsed and put in a list. The matching dataframes are now in the same indices
# between the two lists of dataframes. This means when they are concatenated, all JSON elements will be in corresponding
# indices.

for sql_id, snowflake_id in zip(sql_id_list, snowflake_id_list):
    concat_sql_dataframe_list.append(json_dataset(turn_json_into_dictionary(sql_source_dataframe.query('id == @sql_id').iloc[0])))
    concat_snowflake_dataframe_list.append(json_dataset(json.loads(snowflake_target_dataframe.query('INPUT_KEY == @snowflake_id').iloc[0])))


# Concatenates The SQL DataFrame List Into One DataFrame Of Parsed JSON

In [53]:
# Makes concatenated DataFrame, and adds an index column that will match snowflake concatenated dataframe.
sql_concat_df = pd.concat(concat_sql_dataframe_list).reset_index(drop=True).reset_index()


# Concatenates The Snowfalke DataFrame List Into One DataFrame Of Parsed JSON

In [54]:
# Makes concatenated DataFrame, and adds an index column that will match sql concatenated dataframe.
snowflake_concat_df = pd.concat(concat_snowflake_dataframe_list).reset_index(drop=True).reset_index()


# The Section Below is for the purpose of sorting all columns in independently and adding a new index to each. This results in the data being trully 1:1 between environments when used in the compare_function.

In [55]:
def compare_all_columns(source_df, target_df):
    compare_result_numbers = []
    source_columns = []
    target_columns = []
    compare_result_dfs = []

    for source_column, target_column in zip(source_df.columns, target_df.columns):
        sort_source_df = source_df[source_column].to_frame().sort_values(by=source_column)
        sort_target_df = target_df[target_column].to_frame().sort_values(by=target_column)

        aligned_source = sort_source_df.reset_index(drop=True)
        aligned_target = sort_target_df.reset_index(drop=True)

        results = aligned_source[source_column].compare(aligned_target[target_column])

        source_columns.append(source_column)
        target_columns.append(target_column)
        compare_result_numbers.append(len(results))
        compare_result_dfs.append(results)

    return [compare_result_numbers, compare_result_dfs]


In [56]:
compare_result_numbers = []
source_columns = []
target_columns = []
compare_result_dfs = []

full_sorted_df_source = pd.DataFrame()
full_sorted_df_target = pd.DataFrame()

for source_column, target_column in zip(dbsource.columns, datahubtarget.columns):
    sort_four_columns_source = dbsource[source_column].to_frame().sort_values(by=source_column)
    sort_four_columns_target = datahubtarget[target_column].to_frame().sort_values(by=target_column)

    aligned_source = sort_four_columns_source.reset_index(drop=True)
    aligned_target = sort_four_columns_target.reset_index(drop=True)

    full_sorted_df_source = pd.concat([full_sorted_df_source, aligned_source], axis=1)
    full_sorted_df_target = pd.concat([full_sorted_df_target, aligned_target], axis=1)

    # for key, value in columns_to_convert.items():
    #     if key == aligned_source.columns:
    #         print(source_column)

    #     aligned_source[key] = pd.to_numeric(aligned_source[key], errors='coerce')
    #     aligned_target[key] = pd.to_numeric(aligned_target[key], errors='coerce')

    results = aligned_source[source_column].compare(aligned_target[target_column])

    source_columns.append(source_column)
    target_columns.append(target_column)
    compare_result_numbers.append(len(results))
    compare_result_dfs.append(results)


In [57]:
test_results_df = pd.DataFrame({
    'Source Columns': source_columns,
    'Target Columns': target_columns,
    'Results': compare_result_numbers
})


# This section looks for file in a file path, uses the string from the file name to construct meta data to populate sql scripts and dataframes to then be processed and validated.

In [ ]:
import os
import pandas as pd
import json
from collections import defaultdict

# Assuming 'filepath' is defined elsewhere
# plist = os.listdir(filepath)

letterdict = {}
for file in plist:
    if '.json' in file:
        letter_list = re.findall(r'(\d+_historical)', file)
        letter_result = '_'.join(map(str, letter_list))

        gdict = {}
        for file in plist:
            if 'historical_2024081683200734.csv.gz' in file:
                notincluded = []
                data = pd.read_csv(file, sep=",")
                letter_result = file.split("_")[1].replace("Historical", "")
                result = re.findall(r'\d+', letter_result)
                gdict[int(result[0])] = data

        sorted_gdict = sorted(gdict.items())

grouped_data = defaultdict(list)
columnlist = []
schemalist = []
tablelist = []
testkeylist = []
filelist = []
column_specification_list = []
count = 1

for k, v in enumerate(list(letterdict.keys())):
    sortkeys = letterdict[k]
    print(sortkeys)

    most_rec = sorted_gdict[1][1]
    nrd = pd.read_csv(r'filepath'.format(most_rec), nrows=2)
    column_specification_count = 1

    stg_dict = {}

    for column in nrd.columns:
        for k in keys:
            if k == 'Name':
                stg_dict[k] = column
            elif k == 'sequence':
                stg_dict[k] = str(column_specification_count)
            elif k == 'role' and column_specification_count == 1:
                stg_dict[k] = 'Key'
                column_specification_count += 1
            else:
                stg_dict[k] = 'Value'
                column_specification_count += 1

        stg_json_dump = json.dumps(stg_dict)
        column_specification_list.append(stg_json_dump)

    tablelist.append(k)
    columnlist.append(list(nrd.columns))
    schemalist.append('CreditLens')
    testkeylist.append(k)
    filelist.append(most_rec)
    count += 1

for key, value in zip(tablelist, column_specification_list):
    grouped_data[key].append(value)

for table in tablelist:
    csllist.append(', '.join(grouped_data[table]))

# Metadata script template, the column name and column will have the ordinal position of the new column
# to be tested in the target table after final job run.
